In [3]:
from urllib.request import urlopen
from urllib import parse
from selenium import webdriver
from bs4 import BeautifulSoup

from flask import Flask
from flask import request
from flask import jsonify

import time
import os
import json
import re
import sys, traceback
import threading

KW_CODE =0
KM_CODE =1
DJ_CODE=2
DS_CODE=3
DD_CODE=4
MJ_CODE=5
SY_CODE=6
SM_CODE=7
SK_CODE=8
SW_CODE=9
SS_CODE=10
HS_CODE=11


# In[4]:


base_start_url_list = ['http://kupis.kw.ac.kr/',
'http://lib.kookmin.ac.kr/',
'http://library.daejin.ac.kr/',
'http://discover.duksung.ac.kr/',
'http://library.dongduk.ac.kr/',
'http://lib.mju.ac.kr/',
'http://lib.syu.ac.kr/',
'https://lib.smu.ac.kr/',
'http://library.skuniv.ac.kr/',
'http://lib.swu.ac.kr/',
'http://lib.sungshin.ac.kr/',
'http://hsel.hansung.ac.kr/']

#https://lib.kookmin.ac.kr/#/search/info/si?all=1%7Ck%7Ca%7C9788988474839

base_middle_url_list = ['search/caz/result?st=KWRD&commandType=advanced&si=1&q=',
'#/search/info/si?all=1%7Ck%7Ca%7C',
'',
'',
'',
'',
'',
'Search/?q=',
'',
'',
'',
'']





base_end_url_list = ['&b0=and&weight0=&si=2&q=&b1=and&weight1=&si=3&q=&weight2=&inc=TOTAL&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&lmt0=TOTAL&lmtsn=000000000003&lmtst=OR&rf=&rt=&range=000000000021&cpp=10&msc=100',
'',
'',
'',
'',
'',
'',
'',
'',
'',
'',
''
]



class LibraryLS:
    def __init__(self, sid):
        self.sid = sid
        self.loanStatusList = []
        self.errorMessage= ''
    


# In[8]:


class LoanStatus:
    def __init__(self, title = '', RN = '', CN = '', POS = '', STATE = -1, RDD = '', BN = 0, errorMessage = ''):
        self.title = title
        self.RN= RN
        self.CN= CN
        self.POS= POS
        self.STATE= STATE
        self.RDD = RDD#'2222-12-31'
        self.BN = BN
        self.errorMessage = errorMessage
'''#STATE : 도서 상태,
#-1  에러
#1 대출가능
#0 대출중
#2 지정도서
# 대출불가 아직 생각 안함.
#RN : 등록번호
#CN : 청구기호
#POS : 위치
이런것들'''



stateDict = {'대출중': 0, '이용가능':1,'대출가능':1, '지정도서':2}




In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
options = Options()
options.headless = True
browser = webdriver.Chrome(executable_path="./chromedriver.exe", options=options)

In [4]:
def makeSearchUrl(ISBN, title, sid, flag):
    if flag ==1:
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]
    else :
        # flag 2.
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + title+ base_end_url_list[sid]
    return searchUrl

In [7]:
#https://lib.kookmin.ac.kr/#/search/info/si?all=1%7Ck%7Ca%7C9788988474839
#https://lib.kookmin.ac.kr/#/search/info/si?all=1%7Ck%7Ca%7C%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%20%EC%A0%84%EB%AC%B8%EA%B0%80%20%EA%B0%80%EC%9D%B4%EB%93%9C

In [35]:
ISBN = '9788988474839'
koreaTitle = '데이터 분석 전문가 가이드'
sid = 7
searchFlag= 2
title = parse.quote(koreaTitle)
urlresult = makeSearchUrl(ISBN, title, KM_CODE, searchFlag)

In [36]:
urlresult

'http://lib.kookmin.ac.kr/#/search/info/si?all=1%7Ck%7Ca%7C%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%20%EC%A0%84%EB%AC%B8%EA%B0%80%20%EA%B0%80%EC%9D%B4%EB%93%9C'

In [39]:
browser.get(urlresult)

res = browser.page_source

soup=BeautifulSoup(res, 'html.parser')

In [40]:
soup

<html class="k-webkit k-webkit81" lang="ko"><!--<!(endif)--><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style>
<title>소장자료검색 | 국민대학교 성곡도서관</title>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="./assets/css/ui.css" rel="stylesheet"/>
<link href="./assets/images/km/favicon.ico" rel="shortcut icon"/>
<link href="./assets/css/main.css?sv=2020.4.6.10" rel="stylesheet"/>
<script async="" src="//www.google-analytics.com/analytics.js"></script><script src="./assets/files/jquery-1.11.3.min.js" type="text/javascript"></script>
<script src="./assets/files/nppfs-1.9.0.js" type="text/javascript"></script>
<script src="./assets/files/jQuery.XDomainRequest.js" type=

In [15]:
result1 = soup.find('div', {'ui-view':'pyxis-total-result'}).find_all('div', {'class': 'ikc-search-item'})

AttributeError: 'NoneType' object has no attribute 'find_all'

In [21]:
#result1 = soup.find_all('div', {'class':'ikc-search-result'})
result1 = soup.find_all('div', {'class':'ikc-main'})

In [22]:
len(result1)

1

In [24]:
result1[0]

<div class="ikc-main" ui-view="main"><h2 class="ikc-page-title"><strong>소장자료검색</strong></h2><!-- --><div ui-view="form"><div class="ikc-search-form-wrap"><form class="ng-pristine ng-valid" ng-submit="search()"><div class="ikc-search-topbtns"><span class="ikc-search-opt-wrap"><input class="ng-pristine ng-untouched ng-valid ng-empty" id="research" name="research" ng-model="research" title="결과내 재검색" type="checkbox"/> <a class="ng-hide" href="" id="btn-searchinresult-close" ng-click="research = !research; tabToGo('btn-searchinresult-open')" ng-show="research" title="결과내 재검색 끄기">결과내 재검색 <span class="ikc-search-opt" ng-class="{'ikc-search-opt-on': research}"><span class="ikc-search-opt-on" title="활성상태">ON</span></span></a> <a href="" id="btn-searchinresult-open" ng-click="research = !research;  tabToGo('btn-searchinresult-close')" ng-show="!research" title="결과내 재검색 켜기">결과내 재검색 <span class="ikc-search-opt" ng-class="{'ikc-search-opt-on': research}"><span class="ikc-search-opt-off" title="비활성상

In [25]:
result2 = result1[0].find_all('div', {'class':'ikc-search-result'})

In [26]:
len(result2)

2

In [29]:
result3= result1[0].find_all('div', {'ng-show': 'totalCount'})

In [30]:
len(result3)

1

In [31]:
result4 = result3[0].find_all('div', {'class':'ikc-search-result'})

In [32]:
len(result4)

1

In [41]:
result1 = soup.find('div', {'class':'ikc-main'}).find('div', {'class':'ikc-search-result'})

In [42]:
result1

<div class="ikc-search-result"><!-- --><div ui-view="result"><h4 class="ikc-search-result-title" ng-show="totalCount"><span ng-hide="isFuzzy &amp;&amp; fuzzyShowed">검색결과</span> <span class="ng-hide" ng-show="isFuzzy &amp;&amp; fuzzyShowed">유사어 검색 결과</span></h4><div class="ikc-search-noresult ikc-no-record ng-hide" ng-show="result &amp;&amp; !totalCount"><strong>[{"id":3259950,"biblioType":{"id":1,"name":"단행본","imageMapOffset":11},"thumbnailUrl":"http://image.aladin.co.kr/product/8748/49/cover/898847483x_1.jpg","isbn":"9788988474839","issn":null,"titleStatement":"데이터 분석 전문가 가이드","author":"한국데이터베이스진흥원","publication":"서울 :한국데이터베이스진흥원,2016","etcContent":"{\"PAGE\":\"xxi, 988 p.\",\"CORPORATE_NAME\":\"한국데이터베이스진흥원\",\"HEIGHT\":\"25 cm\"}","locationId":3,"branchVolumes":[{"id":1,"name":"성곡도서관","isCr":false,"volume":"005.74 한17ㄷ2","cState":"대출가능","isSubscribed":false,"hasItem":true}],"erSources":[],"resources":null,"newResources":null,"similars":[{"id":3096305,"titleStatement":"데이터분석 전문가·준전문가 

In [43]:
result2 = result1.find_all('div', {'class':'ikc-search-item'})

In [44]:
len(result2)

4

In [64]:
#result3 = result2[0].find_all('li')
result3 = result2[0].find('div', {'class':'ikc-item-info'}).find('li').find('a')

In [65]:
len(result3)

1

In [62]:
result3[0]

KeyError: 0

In [66]:
result3

<a class="ikc-item-title" href="#/search/detail/3259950" ng-click="goDetail(item.id)" ng-href="#/search/detail/3259950"><span ng-bind-html="item.titleStatement | IKGHighlight:searchKeywords"><highlight>데이터</highlight> <highlight>분석</highlight> <highlight>전문가</highlight> <highlight>가이드</highlight></span></a>

In [67]:
ti = result3.text.strip()
ti

'데이터 분석 전문가 가이드'

In [68]:
result3.get('href')

'#/search/detail/3259950'

In [69]:
urlresult = 'https://lib.kookmin.ac.kr/#/search/detail/3259950'
browser.get(urlresult)
res = browser.page_source
soup=BeautifulSoup(res, 'html.parser')

In [70]:
soup

<html class="k-webkit k-webkit81" lang="ko"><!--<!(endif)--><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style>
<title>소장자료검색 | 국민대학교 성곡도서관</title>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="./assets/css/ui.css" rel="stylesheet"/>
<link href="./assets/images/km/favicon.ico" rel="shortcut icon"/>
<link href="./assets/css/main.css?sv=2020.4.6.10" rel="stylesheet"/>
<script async="" src="//www.google-analytics.com/analytics.js"></script><script src="./assets/files/jquery-1.11.3.min.js" type="text/javascript"></script>
<script src="./assets/files/nppfs-1.9.0.js" type="text/javascript"></script>
<script src="./assets/files/jQuery.XDomainRequest.js" type=